In [ ]:
%run config.py

In [ ]:
import random
def randomgate(qbits):
	op=random.choice(['x','y','z','h','cx','cz'])
	if op=='cx' or op=='cz':
		q1=random.choice(range(qbits))
		q2=random.choice(range(qbits))
		while q1==q2:
			q2=random.choice(range(qbits))
		return 'qc.'+op+'('+str(q1)+','+str(q2)+')', op+' q'+str(q1)+',q'+str(q2)
	elif op=='x' or op=='y' or op=='z' or op=='h':
		q1=random.choice(range(qbits))
		return 'qc.'+op+'('+str(q1)+')', op+' q'+str(q1)
	 

In [ ]:
def randomcircuit(qbits, gates):
	qiscirc=""
	bmcirc=""
	for i in range(gates):
		qiscirct, bmcirct = randomgate(qbits)
		qiscirc+=qiscirct+'\n'
		bmcirc+='\t'+bmcirct+'\n'
	return qiscirc, bmcirc

In [ ]:
def randomsim(qbits,gates):
	qiscirc, bmcirc = randomcircuit(qbits, gates)
	
	with open('qiskit/circuit.py.tmpl','r') as f:
		qiskitscript=f.read()

	with open('bmqsim/program.bmq.tmpl','r') as f:
		bmprogram=f.read()

	qiskitscript=qiskitscript.replace('%%CIRCUIT%%',qiscirc)
	bmprogram=bmprogram.replace('%%CIRCUIT%%',bmcirc)

	with open('qiskit/circuit.py','w') as f:
		f.write(qiskitscript)

	with open('bmqsim/program.bmq','w') as f:
		f.write(bmprogram)

	%run analyze.ipynb

In [ ]:
randomsim(qbits,10)
if passed:
	print('Test passed')
else:
	print('Test failed')